# Описание задачи

Необходимо написать скрипт для парсинга диалогов из файла. 

**Главные задачи, которые должен выполнять скрипт:**
+ Извлекать реплики с приветствием – где менеджер поздоровался. 
+ Извлекать реплики, где менеджер представил себя. 
+ Извлекать имя менеджера. 
+ Извлекать название компании. 
+ Извлекать реплики, где менеджер попрощался.
+ Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

**Результат:** записывается в файл data/result.csv. Информация об отсутствии прощаний и приветствий выводится в пункте 9.

При работе была использована библиотека Spacy.
Приветствия вида 'это <ИМЯ>' не удалось отличить от обычного вопроса, уточняющего имя клиента, так как смысл зависит от интонации.

# Загрузка модели и данных

In [1]:
import pandas as pd
import spacy

In [2]:
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_2629/3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
nlp = spacy.load('ru_core_news_sm')

In [580]:
df = pd.read_csv('data/test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,Ага


отберем вспомагательный датафрейм, который относится только к менеджерам

In [581]:
df_m = df[df.role == 'manager'].sort_values(['dlg_id', 'line_n'])#.reset_index()#.copy()


добавляем новое поле INSIGHT, в которое будем помещать информацию

In [582]:
df_m['insight'] = '' * len(df_m)

In [583]:
df_m.head()

,dlg_id,line_n,role,text,insight
1,0,1,manager,Алло здравствуйте,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,
5,0,5,manager,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать,
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,
11,0,11,manager,Что для вас приоритет,


# Поиск приветствий

Определяем словарь приветсвенных выражений

In [10]:
greet = {}
day_adjective = ['добрый']


# перед существительными не должно быть главного глагола
greet['NOUN'] = {}
greet['NOUN']['Nom'] = {
        'привет': {}, 
        'утро': {'place_any_cl' : day_adjective},
        'вечер': {'place_any_cl' : day_adjective},
        'день': {'place_any_cl' : day_adjective},
        'время':{'place_left_cl': day_adjective, 'place_right_cl': ['суток']}, 
        
}
# библиотека не правильно определяет падеж слова 'день'
greet['NOUN']['Acc'] = {

        'день': {'place_any_cl' : day_adjective},
        'здравие':{'place_right_h': ['желать']}
              
}
    
greet['NOUN']['Gen'] = {
        'ночь': {'place_any_cl' : day_adjective},
              
}

greet['VERB'] = {'здравствуй':{}, 'здравствовать':{},  'приветствовать': {}}

In [11]:
def val_in_list(search_val, doc_text_ch, doc_text_head):
    #print(doc_text_head)
    if doc_text_ch:
        for doc in doc_text_ch:
            if doc.lemma_ in search_val:
                return True
    if doc_text_head:
        if doc_text_head.lemma_ in search_val:
            return True
    return False

In [12]:
def is_greeting(token):
    greet_flag = False
    #если часть речи текущего слова есть в словаре приветствий
    if token.pos_ in greet:
        gr = greet[token.pos_]
        
        # сначала анализируем приветствия-существительные
        if token.pos_ == 'NOUN':
            case = token.morph.get('Case')[0]
            # сначала обработаем выражения исключения:
            # 1. 'Здравия желаю' и его формы
            if token.lemma_ == 'здравие':
                if case in gr and token.lemma_ in gr[case]:
                    greet_flag = val_in_list(gr[case][token.lemma_]['place_right_h'], token.children, token.head)
                return greet_flag
 
            
            # теперь обрабатываем более общие формы
            if case in gr and token.lemma_ in gr[case]:
                tec_gr = gr[case][token.lemma_]
                #print(tec_gr)

                if 'place_any_cl' in tec_gr:
                    greet_flag = val_in_list(tec_gr['place_any_cl'], token.children, token.head)

                if 'place_left_cl' in tec_gr:
                    greet_flag = val_in_list(tec_gr['place_left_cl'], token.lefts, token.head)

                if 'place_right_cl' in tec_gr:
                    greet_flag = val_in_list(tec_gr['place_right_cl'], token.rights, token.head)
                    
                if len(tec_gr) == 0:
                    greet_flag = True
                    
                    
        # теперь анализируем приветсвия-глаголы
        elif token.pos_ == 'VERB':
            if token.lemma_ in gr:
                greet_flag = True
                    
    return greet_flag


# Поиск имени автора

In [415]:
verb_for_autor = {'писать', 'звонить', 'общаться', 'обращаться', 'разговаривать', 'спрашивать', 'отвечать', 'говорить'}

In [416]:
def get_other_part_of_name(token):
    res = ''
    for ch in token.children:
        if ch.dep_ == 'flat:name':
            res += ' ' + ch.text#.capitalize()
    return res
'''
def my_name_is_rec(token):
    res = ''

    if token.head == token: 
        return res 
    if token.dep_ in ['xcomp', 'nsubj'] and token.head.lemma_ == 'звать':
        
        fl = 0
           # здесь проверяем является ли слово именем
        for ch in token.head.children:
            if ch.lemma_ != token.lemma_:
                if ch.dep_ == 'obj' and ch.lemma_ == 'меня':
                    
                    res += token.text
                    fl = 1
                    break
            # теперь смотрим есть ли еще слова относящиеся к имени
        if fl:
            #print('res:', res)
            res += get_other_part_of_name(token)
        return res
    else:
        
        return token.text + ' ' + my_name_is_rec(token.head)
'''    

def my_name_is_rec(token):

    if token.dep_ in ['xcomp', 'nsubj', 'obj'] and token.head.lemma_ == 'звать':
           # здесь проверяем является ли слово именем
        for ch in token.head.children:
            if ch.lemma_ != token.lemma_:
                if ch.dep_ == 'obj' and ch.lemma_ == 'меня':
                    return True
    return False
            
 
    
def get_propn(token):
    
    propn = 0
    
    # при написании маленькой буквы не всегда распознается как имя собственное
    if token.pos_ != 'PROPN':
        cap = nlp(token.lemma_.capitalize())
       
        if cap[0].pos_ == 'PROPN':
            propn = 1
    else: 
        propn = 1
    return propn
    
def is_name(token):

    res = ''
    propn = get_propn(token)
    
   
    if propn:
       
        #---------------выискиваем фразы шаблона 'Меня зовут ...'----------------
        
        
        if my_name_is_rec(token):
            res = token.text + get_other_part_of_name(token)
            
            return res
       
        # ------------------------фразы шаблона 'Я ... ''---------------------------
        
        if token.dep_ in ['appos', 'punct'] and token.head.lemma_ == 'я':   
         
            res = token.text + get_other_part_of_name(token)
            return res#.capitalize()
        elif token.pos_ == 'ADJ':
            for ch  in token.children:
                if ch.lemma_ == 'я' and ch.dep_ == 'nsubj':
                    res = token.text + get_other_part_of_name(token)
                    return res#.capitalize()
        # ----------------------- фразы шаблона 'Мое имя ...'----------------------
   
        
        if token.dep_ in ['appos', 'nmod'] and token.head.lemma_ == 'имя': 
        
            for ch in token.head.children:
           
                if ch.dep_ == 'det' and ch.lemma_ == 'мой':
                    res = token.text + get_other_part_of_name(token)
                    return res#.capitalize()
                
           
        for ch in token.children:
           
            if ch.dep_ in ['obj', 'nsubj'] and ch.lemma_ == 'имя':
                
                for ch_name in ch.children:
                  
                    if ch_name.dep_ == 'det' and ch_name.lemma_ == 'мой':  
                      
                        res = token.text + get_other_part_of_name(token)
                        return res#.capitalize()
                    
        #------------------------фразы типа 'тревожит/беспокоит/пишет/.. ... '
        if token.head.pos_ == 'VERB' and token.dep_ == 'nsubj':
            if token.head.lemma_ not in verb_for_autor:
                return res
         
            #считаем, что у глагола только один ребенок со свзяью nsubj
            fl = 0
            for ch_name in token.head.children:
                if ch_name.dep_ == 'nsubj' and ch_name.lemma_ != token.lemma_:
                    return ''
      
                if ch_name.dep_ == 'iobj' and ch_name.pos_ == 'PRON':  
                    fl = 1  
                    
            if fl:
                res = token.text + get_other_part_of_name(token)
        return res

# Поиск названия компании

In [14]:
lst_syn_company = ['компания', 'организация', 'фирма']

In [351]:
def company_name_is_rec(token, orig_token_lemma):
    res = False

    # здесь проверяем является ли слово компанией
    k = 0
    for ch in token.children:
        k = 1
        
        if ch.lemma_ != token.lemma_ and ch.lemma_ != orig_token_lemma:
            if ch.lemma_ == 'компания':
                return True
            else:
                if company_name_is_rec(ch, orig_token_lemma):
                    return True
                
        
    return res
    
def get_all_name_company(token):
   
    res = [ch.text for ch in token.children if ch.dep_ == 'obj']
  
    return ' '.join(res)
        
    
def get_name_company(token):
   
    res = ''
    #propn = get_propn(token)
    # это компания ...
  
    if token.dep_ in ['appos', 'nsubj'] and token.head.lemma_ in lst_syn_company:
        
        return token.text + ' ' + get_all_name_company(token)
    # пишу из компании ...
    if token.dep_ in ['punct'] and token.head.pos_ == 'VERB':
        for ch in token.head.children:
            
            if ch.lemma_ in lst_syn_company:
                return token.text + ' ' + get_all_name_company(token)
   
    # это анжела компания ...
    for ch in token.children:
        
        if ch.dep_ == 'nsubj' and ch.lemma_ in lst_syn_company:
            for ch_comp in ch.children:
                if ch_comp.pos_ == 'PROPN':
                    return res
            return token.text + ' ' + get_all_name_company(token)
    # Вас беспокоит анжела компания ...
    if token.dep_ in ['conj', 'nsubj']:
        if company_name_is_rec(token.head, token.lemma_):
            return token.text + ' ' + get_all_name_company(token)

        
    
        

# Поиск прощаний

In [16]:
# при прощании слова в родительном падежа + добавить лемму 'время суток'
farewell = {}
farewell['ADJ'] = ['хороший', 'добрый', 'спокойный']
time_of_day = ['утро', 'день', 'вечер', 'ночь', 'время', 'всего']


In [503]:

def rec_goodbye(token):
 
    res = False

    k = 0
    for ch in token.children:
        k = 1
        
        if ch.pos_ == 'PRON' and ch.lemma_ in time_of_day:
          
            return True
        else:
             if(rec_goodbye(ch)):
                    return True
                
               
    return res


def is_goodbye(token):

    bye_flag = False
 
    # до свидания
    if token.lemma_ == 'свидание':
        for ch in token.children:
            if ch.dep_ in ['case','ADP'] and ch.lemma_ == 'до':
                return True
        
    # прощай/ прощайте
   
    elif token.lemma_ == 'прощать':
        if token.text in ['прощай', 'прощайте']:
            #обрабатываем ситауции 'прощайте его/их/соседа/....'
            for ch in token.children:
                if ch.dep_ in ['obl','obj']:# and ch.pos_ == 'PRON'
                    return False
            return True
        
    # до завтра/вечера/....    
    elif token.lemma_ == 'до':
        if token.dep_ == 'case' and token.head.pos_ in ['NOUN', 'ADV', 'ADJ'] and token.head.head == token.head:
            #if token.head.dep_ == 'obl' and token.head.head.pos_ in ['VERB']:
            return True
        else:
            return False
    # доброго/хорошего/... дня/вечера/..
    # прощания - прилагательные в род.падеже
    elif token.pos_ == 'ADJ' and token.lemma_ in farewell['ADJ'] and len(token.morph.get('Case')) > 0 and token.morph.get('Case')[0] == 'Gen':
        # хорошего/... времени суток
        
        if  token.head.lemma_ in time_of_day:
            if token.head.lemma_ == 'время':
                for ch in token.head.children:
                    if ch.text == 'суток':
                        return True
                return False
            else:
                return True

        # ситуации типа 'тогда всего доброго'
        for ch in token.children:
            
            if ch.dep_ == 'advmod':
                for ch_ch in ch.children:
                    if ch_ch.pos_ == 'PRON' and ch_ch.lemma_ in time_of_day:
                        return True
        
        if token.dep_ == 'amod':
            return rec_goodbye(token.head)
            
    
    return False
    
                

# Обход всех строк

In [389]:
def per_or_org(pred, tec, fl_pers = 1, fl_comp = 1):
    '''
    Функция определяющая является ли токе именем собственным или нет
    
    fl_pers = 1 если ищем сотрудника, = 0 если нет
    fl_comp = 1 если ищем компанию , = 0 если нет 
    
    '''
    res = {}
    tec_doc = nlp(pred.text + ' ' + tec.text.capitalize())

    if tec_doc[1].ent_type > 0:

        if fl_pers:# and tec_doc[1].ent_type_ == 'PER':
            per = is_name(tec)
            if per:
                return {'person': per}
            
        elif fl_comp: # and tec_doc[1].ent_type_ in ['ORG']:
            org = get_name_company(tec)
            
            if org:
                return {'company' : org}
    return res


def search_info(text):
    
    res = {}
    
    doc = nlp(text)

    #обходим по всем словам теста, в тексте каждую сущность ищем только один раз
    for i, token in enumerate(doc):
        
        if token.pos_ == 'NUM':
            continue
    
        #print('==========',token.text,'===========')
        
        fl_search_pers = 0 if 'person' in res else 1
        fl_search_comp = 0 if 'company' in res else 1
        fl_search_greet = 0 if 'greeting' in res else 1
        fl_search_goodbye = 0 if 'goodbye' in res else 1
      
        

        # флажок, указывающий на то, что мы нашли уже принадлежность этого токена к одной из категорий
        fl_find = {}

        ##################существительные и глаголы#################
        if token.pos_ in ['NOUN', 'VERB'] :
            if is_greeting(token):
                fl_find = {'greeting': True}
            else:
                if is_goodbye(token) :
                    fl_find = {'goodbye': True}

                else:
                    if i > 0:
                        
                        fl_find = per_or_org(doc[i-1], token, fl_search_pers, fl_search_comp)
                        
        ##################имена собственные#################   
        elif token.pos_ == 'PROPN':
            
            if i > 0:
                
                fl_find = per_or_org(doc[i-1], token, fl_search_pers, fl_search_comp)
        ##################прилагательные#################
        elif token.pos_ == 'ADJ':
            if fl_search_pers:
                per = is_name(token)
                if per:
                    fl_find = {'person': per}
                    
                    
            if fl_search_goodbye and len(fl_find) == 0:
                if is_goodbye(token) :
                    
                    fl_find = {'goodbye': True}
                    
        ##################предлоги#################    
        elif token.pos_ == 'ADP':
            if fl_search_goodbye:
                # обработка прощаний типа 'до завтра/вечера/...'
                if token.lemma_ == 'до':
                    if is_goodbye(token) :
                        fl_find = {'goodbye': True}
    
        if len(fl_find):
            res.update(fl_find)
            
        
    
    return res


                

# Применение функций к датафрейму

In [585]:
df_m['insight'] = df_m.apply(lambda row: str(search_info(row.text)), axis=1)

In [589]:
df_m[df_m.dlg_id == 3]

,dlg_id,line_n,role,text,insight
250,3,1,manager,Алло дмитрий добрый день,{'greeting': True}
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,"{'person': 'максим', 'company': 'китобизнес '}"
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подсказала что у вас есть какие то открытые вопросы связанные с техническими особенностями а мы серым вот готов буду его ответить или взять паузу и решите эти вопросы уточните пожалуйста в чем задача состоит,{}
256,3,7,manager,Угу,{}
260,3,11,manager,Очень важно я думаю вот например да,{}
265,3,16,manager,Да,{}
266,3,17,manager,Да абсолютно все то есть это единое такое окно коммуникации в которое подтягивается информация со всех каналов коммуникации в том числе вы там видите письма и сообщения из разных мессенджеров доступны к ответу,{}
268,3,19,manager,Так а вот подскажите пожалуйста может быть просто мы вам подскажем какие то еще продукты от нашей компании которые можно было бы внедрить и получить какую то пользу вот у вас вообще бизнес вы на чем специализируете и с чем связано бизнес,{}
273,3,24,manager,Транспортная компания хорошо а у вас уже вот все интеграции сделаны ну то есть там каналы коммуникации от клиентов поступления ледов вот это вот уже сделано или это вы еще только планируете все настраивать,{}
275,3,26,manager,Да но если мы будем прям настраивать вам серым систему внедрять так скажем да то это конечно не будет бесплатным вот просто здесь я вот услышал что вы какие то вещи будете сами делать ну допустим там настраивать воронки там да и так далее просто вам там в перспективе возможно потребуется какая то помощь все равно там настройки тех или иных интеграций то есть вы больше времени потратите нежели там через интегратора,{}


# Ищем диалоги без приветствий и прощаний

In [587]:
for dlg in df_m.dlg_id.unique():
    gr_flag = 0
    bye_flag = 0
    # выбираем поля где есть insight
   # print(df_m[(df_m.dlg_id == dlg) and (df_m.insight != '{}')].text)
    for dl_info in df_m[(df_m.dlg_id == dlg) & (df_m.insight != "{}")].insight:
        
        dl_info = dl_info.strip('{}')

        for el in dl_info.split(','):

            if el.split(':')[0].strip("'") == 'greeting':
  
                gr_flag = 1
            elif el.split(':')[0].strip("'") == 'goodbye':
                bye_flag =1
            if bye_flag and gr_flag:
                break
                
    if bye_flag and gr_flag:
        continue
    else:
        err_info = []
        if not gr_flag:
            err_info.append('приветствия')
        if not bye_flag:
            err_info.append('прощания')
        print('Для диалога id =', dlg, 'отсутсвуют:', ', '.join(err_info))
        
        
    

Для диалога id = 2 отсутсвуют: прощания
Для диалога id = 4 отсутсвуют: приветствия
Для диалога id = 5 отсутсвуют: приветствия


# Модификация основных данных и сохранение в файл

In [592]:

df = df.merge(df_m[['dlg_id', 'line_n', 'insight']], on=['dlg_id', 'line_n'], how='left')
df.head()


,dlg_id,line_n,role,text,insight
0,0,0,client,Алло,NaN
1,0,1,manager,Алло здравствуйте,{'greeting': True}
2,0,2,client,Добрый день,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,"{'person': 'ангелина', 'company': 'диджитал бизнес'}"
4,0,4,client,Ага,NaN


In [593]:
df.to_csv('data/result.csv')

# Построение графиков (информационно)

In [478]:
from spacy import displacy
text = 'Угу все хорошо да понедельника тогда всего доброго' #компания диджитал бизнес
#text = 'спокнойной ночи' 

doc = nlp(text)
displacy.render(doc, style='dep', jupyter=True)
for token in doc:
    print(token.lemma_, token.dep_, token.pos_, token.head, token.ent_type_)


угу ROOT VERB Угу 
все obj PRON Угу 
хороший advmod ADJ Угу 
да advmod PART Угу 
понедельник obj NOUN Угу 
тогда advmod ADV понедельника 
всего obl PRON тогда 
добрый amod ADJ Угу 


Определяем словарь прощальных выражений